# EDA

# ML

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
df_games = pd.read_parquet('games.parquet')
df_items = pd.read_parquet('items.parquet')
df_reviews = pd.read_parquet('Reviews.parquet')
df_generos = pd.read_parquet('generos.parquet')

In [4]:
df_steam = pd.merge(df_games, df_generos, left_on='id', right_on='id')
df_steam.head(2)

,title,release_date,price,id,developer,genres
0,lost summoner kitty,2018-01-04,4.99,761140,kotoshiro,action
1,lost summoner kitty,2018-01-04,4.99,761140,kotoshiro,casual


In [5]:
df_steam = df_steam[['id', 'title', 'genres']]
df_steam.head(2)

,id,title,genres
0,761140,lost summoner kitty,action
1,761140,lost summoner kitty,casual


In [6]:
df_steam = df_steam.drop_duplicates()
df_steam.head(2)

,id,title,genres
0,761140,lost summoner kitty,action
1,761140,lost summoner kitty,casual


In [6]:
# Agrupa los géneros y títulos para cada juego en una lista
grouped_genres_title = df_steam.groupby('id')[['genres', 'title']].agg(list).reset_index()

# Crea una nueva columna que contiene la lista de géneros como texto
grouped_genres_title['genres_text'] = grouped_genres_title['genres'].apply(lambda x: ' '.join(x))

# Crea una matriz TF-IDF para los géneros de los juegos
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=500)  # Cambia 500 al número deseado de características
tfidf_matrix = tfidf_vectorizer.fit_transform(grouped_genres_title['genres_text'])

# Calcula la similitud del coseno entre los juegos
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [7]:
df=grouped_genres_title

In [8]:
# Función para obtener recomendaciones
def recomendacion_juego(id):
    if id not in df['id'].values:
        no_encontrado = f'El id {id} no fue encontrado en la base de datos'
        return no_encontrado
    
    else:
        # Encuentra el índice del juego con el ID dado
        idx = df[df['id'] == id].index[0]

        # Obtén las puntuaciones de similitud del coseno para todos los juegos
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Ordena los juegos en función de las puntuaciones de similitud
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Obtén los índices de los juegos más similares (excluyendo el juego de entrada)
        sim_scores = sim_scores[1:6]  # Cambia esto si quieres más o menos recomendaciones

        # Obtiene los títulos de los juegos recomendados
        game_indices = [i[0] for i in sim_scores]
        recommended_game_titles = df.iloc[game_indices]['title']

        # Convierte la lista de títulos en una lista plana y única
        recommended_game_titles_flat = list(set([title for sublist in recommended_game_titles for title in sublist]))

        return recommended_game_titles_flat

In [7]:
# Agrupa los géneros y títulos para cada juego en una lista
grouped_genres_title = df_steam.groupby('id')[['genres', 'title']].agg(list).reset_index()

# Crea una nueva columna que contiene la lista de géneros como texto
grouped_genres_title['genres_text'] = grouped_genres_title['genres'].apply(lambda x: ' '.join(x))





In [8]:
# Función para obtener recomendaciones de juego
def recomendacion_juego(id):
    if id not in df['id'].values:
        no_encontrado = f'El id {id} no fue encontrado en la base de datos'
        return no_encontrado
    
    else:
        # Crea una matriz TF-IDF para los géneros de los juegos
        tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=500)  # Cambia 500 al número deseado de características
        tfidf_matrix = tfidf_vectorizer.fit_transform(grouped_genres_title['genres_text'])
        
        # Calcula la similitud del coseno entre los juegos bajo demanda
        cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

        # Encuentra el índice del juego con el ID dado
        idx = df[df['id'] == id].index[0]

        # Obtén las puntuaciones de similitud del coseno para todos los juegos
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Ordena los juegos en función de las puntuaciones de similitud
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Obtén los índices de los juegos más similares (excluyendo el juego de entrada)
        sim_scores = sim_scores[1:6]  # Cambia esto si quieres más o menos recomendaciones

        # Obtiene los títulos de los juegos recomendados
        game_indices = [i[0] for i in sim_scores]
        recommended_game_titles = df.iloc[game_indices]['title']

        # Convierte la lista de títulos en una lista plana y única
        recommended_game_titles_flat = list(set([title for sublist in recommended_game_titles for title in sublist]))

        return recommended_game_titles_flat

In [9]:
recomendacion_juego('772540')

['enforcer: police crime action',
 'thehunter: primal',
 'beyond space remastered edition',
 'the tomorrow war',
 'star trek™ : 25th anniversary']

In [10]:
df.to_parquet('Recomendacion.parquet', index = False)

In [11]:
import gzip

# Suponiendo que cosine_sim es una matriz NumPy
with gzip.open('cosine_sim.npy.gz', 'wb') as f:
    np.save(f, cosine_sim)